In [2]:
import os
import time
import re
import json

In [3]:
#transcription files for train and test
with open("/scratch/work/palp3/MUCS/dataset/train/transcripts/text") as f:
    train_txt= f.read()
with open("/scratch/work/palp3/MUCS/dataset/test/transcripts/text") as f:
    test_txt= f.read()

In [16]:
import math
import re
import numpy as np
import pandas as pd
import torch
import torchaudio
import json
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass
from datasets import Dataset, load_metric, load_dataset, Audio
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
train_txt = train_txt.lower()
test_txt = test_txt.lower()

train_txt = re.sub('\s*\t+\s*', ' ', train_txt)
test_txt = re.sub('\s*\t+\s*', ' ', test_txt)


In [6]:
test_trans = re.findall(r'\d+_[a-zA-Z0-9]+_\d+ (.*)\n',test_txt)
train_trans = re.findall(r'\d+_[a-zA-Z0-9]+_\d+ (.*)\n',train_txt)
print(len(test_trans), len(train_trans))

3136 52825


In [66]:
# only getting the transcription IDs
test_trans_ids = re.findall(r'(\d+_[a-zA-Z0-9]+)_\d+ .*\n',test_txt)
print(test_trans_ids)
#sorted_trans_ids = sorted(test_trans_ids)
#assert test_trans_ids == sorted_trans_ids

['103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', '103085_w5Jyq3XMbb3WwiKQ', 

In [81]:
sorted_trans_ids = sorted(test_trans_ids) ##this shows that the ids are already in the sorted order with the transcripts
assert test_trans_ids == sorted_trans_ids

In [70]:
sorted_trans_ids[-1]

'957491_4xyIm2P6Xzlin341'

In [79]:

# Path to your test data
train_data_path = "/scratch/work/palp3/MUCS/dataset/train_segmented/" #path to the segmented audio files
test_data_path = "/scratch/work/palp3/MUCS/dataset/test_segmented/"

# Read and preprocess audio files
def load_audio(file_path): #note that they are already have a sampling rate of 16KHz so no need to resample
    speech, _ = sf.read(file_path)
    return speech


# Get all WAV files from the test data directory
test_wav = [os.path.join(test_data_path, f) for f in os.listdir(test_data_path) if f.endswith('.wav')]
train_wav = [os.path.join(train_data_path, f) for f in os.listdir(train_data_path) if f.endswith('.wav')]
test_wav_ids = [f for f in os.listdir(test_data_path) if f.endswith('.wav')]
train_wav_ids = [f for f in os.listdir(train_data_path) if f.endswith('.wav')]
test_wav.sort()
train_wav.sort()
test_wav_ids.sort()
train_wav_ids.sort()
# Load all audio files
##test_audio_data = [load_audio(file) for file in test_files]

In [78]:
test_wav.sort()
test_wav_ids.sort()
print(test_wav[-100])
print(sorted_trans_ids[-100])
print(test_wav_ids[-100])


/scratch/work/palp3/MUCS/dataset/test_segmented/957491_4xyIm2P6Xzlin341_0069.wav
957491_4xyIm2P6Xzlin341
957491_4xyIm2P6Xzlin341_0069.wav


In [61]:
sorted_test_wav_ids

'957491_4xyIm2P6Xzlin341_0074.wav'

In [8]:
train_wav_tmp = ["./train_segmented/" + s for s in train_wav_ids]
test_wav_tmp = ["./test_segmented/" + s for s in test_wav_ids]

In [9]:
path_to_train = "/train_segmented/"
path_to_test = "/test_segmented/"

In [80]:
print(len(test_wav), len(train_wav))

3136 52825


In [85]:
def create_sub_dataset(audio_ids, audio_paths, trans):
    dataset={}
    dataset["audio_id"] = audio_ids
    dataset["audio_paths"] = audio_paths
    dataset["transcriptions"] = trans
    return dataset

In [86]:
#creating a dictionary of dataset with audio IDs, corresponding audio paths and transcriptions
dataset_dict = {"train": create_sub_dataset(train_wav_ids, train_wav, train_trans), "test": create_sub_dataset(test_wav_ids, test_wav, test_trans)}

In [80]:
dataset_dict["train"].keys()

dict_keys(['audio_id', 'audio_paths', 'transcriptions'])

In [87]:
import json

out_file = open("MUCS_train_test_dataset_dict_v2.json", "w") #your path to write

json.dump(dataset_dict, out_file, indent = 6)

out_file.close()

In [7]:
vocab_list = list(set(train_txt) | set(test_txt))
vocab_list.remove("\n")


In [8]:
print(len(vocab_list))
print(vocab_list)

147
['ॉ', 'q', '!', '+', '9', '→', '°', '−', 'ह', '०', 'f', 'r', '@', 'w', 'k', 'z', 'ठ', '*', '८', 's', 'ि', 'े', '5', '=', 'ऐ', '१', '९', '×', '%', 'λ', 'त', '2', 'ट', 'ख', 'ी', '8', 'ू', 'श', 'औ', 'ω', '›', 'v', '^', 'ा', '_', '/', '>', 'y', 'द', 'φ', 'ग', '–', 'ॅ', 'ो', 'x', '6', 'उ', '`', 't', 'र', ':', 'ञ', '$', '7', '4', ';', 'g', 'ऊ', '\\', '्', 'छ', 'थ', 'ढ', 'u', 'ै', 'अ', 'ए', 'घ', '&', ' ', '{', '४', '⅓', 'o', 'n', 'ब', 'e', 'p', 'झ', 'ओ', 'i', '…', 'º', 'ल', 'ष', '3', '़', '0', '३', 'l', 'ई', 'π', 'a', 'ड', '~', '६', 'b', 'ँ', 'ऩ', 'इ', '<', '•', 'h', 'ः', 'ौ', 'ण', 'ज', 'क', 'भ', 'j', 'ऑ', '५', 'म', 'ृ', 'च', '२', '#', 'ऍ', 'य', 'c', 'स', 'ं', 'व', 'ु', '1', 'm', '}', '७', 'ऱ', 'प', 'd', 'फ', 'μ', 'ऋ', 'न', 'ध', 'आ']


In [10]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict["<s>"] = len(vocab_dict)
vocab_dict["</s>"] = len(vocab_dict)
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
print(len(vocab_dict))
print(vocab_dict)

151
{'!': 1, '#': 2, '$': 3, '%': 4, '&': 5, '*': 6, '+': 7, '/': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, ';': 20, '<': 21, '=': 22, '>': 23, '@': 24, '\\': 25, '^': 26, '_': 27, '`': 28, 'a': 29, 'b': 30, 'c': 31, 'd': 32, 'e': 33, 'f': 34, 'g': 35, 'h': 36, 'i': 37, 'j': 38, 'k': 39, 'l': 40, 'm': 41, 'n': 42, 'o': 43, 'p': 44, 'q': 45, 'r': 46, 's': 47, 't': 48, 'u': 49, 'v': 50, 'w': 51, 'x': 52, 'y': 53, 'z': 54, '{': 55, '}': 56, '~': 57, '°': 58, 'º': 59, '×': 60, 'λ': 61, 'μ': 62, 'π': 63, 'φ': 64, 'ω': 65, 'ँ': 66, 'ं': 67, 'ः': 68, 'अ': 69, 'आ': 70, 'इ': 71, 'ई': 72, 'उ': 73, 'ऊ': 74, 'ऋ': 75, 'ऍ': 76, 'ए': 77, 'ऐ': 78, 'ऑ': 79, 'ओ': 80, 'औ': 81, 'क': 82, 'ख': 83, 'ग': 84, 'घ': 85, 'च': 86, 'छ': 87, 'ज': 88, 'झ': 89, 'ञ': 90, 'ट': 91, 'ठ': 92, 'ड': 93, 'ढ': 94, 'ण': 95, 'त': 96, 'थ': 97, 'द': 98, 'ध': 99, 'न': 100, 'ऩ': 101, 'प': 102, 'फ': 103, 'ब': 104, 'भ': 105, 'म': 106, 'य': 107, 'र': 108, 'ऱ': 109, 'ल': 110, 'व

In [71]:
#missing_characters = ['आ', 'अं', 'अः', 'ङ', 'क्ष', 'त्र', 'ज्ञ', 'ा', 'ॄ', 'ॅ', 'ॉ', 'ॐ', 'ऽ']
hindi_characters = [
    # Vowels (स्वर)
    'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 'अं', 'अः',
    
    # Consonants (व्यंजन)
    'क', 'ख', 'ग', 'घ', 'ङ',
    'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण',
    'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म',
    'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह',
    
    # Vowel Signs (मात्राएं)
    'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', 'ं', 'ः', 'ँ',
    
    # Numerals (संख्या)
    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९',
    
    # Additional Characters
    '़'  # Nukta
]
 # skipping 'ऽ' as it's used in sanskrit

print(set(hindi_characters)-set(vocab_list))

{'अं', 'अः', 'ङ'}


In [11]:
with open('/scratch/elec/puhe/p/palp3/MUCS/mucs_language_segregated_data/vocab2_MUCS.json', 'w') as vocab_file: #your path here to write
    json.dump(vocab_dict, vocab_file)

In [10]:
#get vocab of both test and train and make a combined vocab list
#add a padding token for the blank token in CTC algo, along with the UNK tag
#then for tokenization use this
#from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab1_MUCS.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [4]:
tokenizer

Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=149, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	147: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	148: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	149: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	150: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [11]:
#rom transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


In [12]:
#from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [6]:
def read_audio_data(file):
    speech_array, sampling_rate = torchaudio.load(file, normalize = True)
    return speech_array, sampling_rate

def resample(waveform):
    transform = torchaudio.transforms.Resample(ORIG_SAMPLING_RATE, TGT_SAMPLING_RATE)
    waveform = transform(waveform)
    return waveform[0]
def process_speech_audio(speech_array, sampling_rate):
    input_values = processor(speech_array, sampling_rate = sampling_rate).input_values
    return input_values[0]

def process_target_text(target_text):
    with processor.as_target_processor():
        encoding = processor(target_text).input_ids

    return encoding

In [30]:
speech_array_check, sampling_rate_check = torchaudio.load('/scratch/work/palp3/MUCS/dataset/train_segmented/772743_KcSRhlKicPd6dvJS_0046.wav', normalize = True)

In [31]:
print(sampling_rate_check)

16000


In [22]:
print(type(dataset_dict["train"]["audio_paths"]))

<class 'list'>


In [23]:
dataset_dict["train"]["audio_paths"][0]

'/scratch/work/palp3/MUCS/dataset/train_segmented/772743_KcSRhlKicPd6dvJS_0046.wav'

In [24]:
path_to_train = "/train_segmented/"
path_to_test = "/test_segmented/"

In [9]:
f = open('MUCS_train_test_dataset_dict.json') 
   
# returns JSON object as  
# a dictionary 
data = json.load(f) 
   


In [129]:
len(train["train"])

52825

In [108]:
try_pd = pd.read_json("MUCS_train_test_dataset_dict.json")
dataset = Dataset.from_pandas(try_pd)

In [110]:
dataset.features

{'train': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'test': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='string', id=None)}

In [25]:
from datasets import load_dataset

train = load_dataset("json", data_files="MUCS_train_test_dataset_dict_v2.json", field="train", cache_dir= "/scratch/work/palp3/MUCS/dataset/")["train"] 
test = load_dataset("json", data_files="MUCS_train_test_dataset_dict_v2.json", field="test", cache_dir="/scratch/work/palp3/MUCS/dataset/")["train"]
#same cache file also available at the accessible location: /scratch/elec/puhe/p/palp3/MUCS/MUCS_train_test_dataset_dict_v2.json


In [23]:
train

Dataset({
    features: ['audio_id', 'audio_paths', 'transcriptions'],
    num_rows: 52825
})

In [21]:
print(type(train), type(train["train"]))

<class 'datasets.dataset_dict.DatasetDict'> <class 'datasets.arrow_dataset.Dataset'>


In [13]:
train["train"] = train["train"].shuffle(seed=42).select(range(200))

In [14]:
train["train"]

Dataset({
    features: ['audio_id', 'audio_paths', 'transcriptions'],
    num_rows: 200
})

In [11]:
x = test["train"].shuffle(seed=42).select(range(200)) #for some reason, although it is correctly using the test split, it is able to be accessed via "train" in test

In [12]:
x

Dataset({
    features: ['audio_id', 'audio_paths', 'transcriptions'],
    num_rows: 200
})

In [ ]:
import pandas as pd

In [29]:
data_df = pd.read_json('/scratch/work/palp3/MUCS/dataset/MUCS_train_test_dataset_dict.json') #accessible location: /scratch/elec/puhe/p/palp3/MUCS/MUCS_train_test_dataset_dict_v2.json


In [48]:
data_df["test"]

audio_id          [957491_4xyIm2P6Xzlin341_0074.wav, 957491_4xyI...
audio_paths       [/scratch/work/palp3/MUCS/dataset/test_segment...
transcriptions    [लिबर ऑफिस impress में एक प्रस्तुति document ब...
Name: test, dtype: object

In [50]:
print(data_df["test"].keys)

<bound method Series.keys of audio_id          [957491_4xyIm2P6Xzlin341_0074.wav, 957491_4xyI...
audio_paths       [/scratch/work/palp3/MUCS/dataset/test_segment...
transcriptions    [लिबर ऑफिस impress में एक प्रस्तुति document ब...
Name: test, dtype: object>


In [14]:
train

DatasetDict({
    train: Dataset({
        features: ['audio_id', 'audio_paths', 'transcriptions'],
        num_rows: 52825
    })
})

In [15]:
test

DatasetDict({
    train: Dataset({
        features: ['audio_id', 'audio_paths', 'transcriptions'],
        num_rows: 3136
    })
})

In [6]:
test_waud = test["train"].cast_column("audio_paths",  Audio(sampling_rate=16000))
train_waud = train["train"].cast_column("audio_paths",  Audio(sampling_rate=16000))


In [18]:
###this works. WIth the cast, there is an audio associated with each file
train_waud = train["train"].cast_column("audio_paths", Audio())
test_waud = test["train"].cast_column("audio_paths", Audio())


In [7]:
test_waud.features

{'audio_id': Value(dtype='string', id=None),
 'audio_paths': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'transcriptions': Value(dtype='string', id=None)}

In [9]:
test_waud[100]

{'audio_id': '103725_UO5ZLDtkc8QMi9Hk_0023.wav',
 'audio_paths': {'path': '/scratch/work/palp3/MUCS/dataset/test_segmented/103725_UO5ZLDtkc8QMi9Hk_0023.wav',
  'array': array([-5.49316406e-04, -6.71386719e-04, -6.10351562e-05, ...,
         -3.05175781e-05,  0.00000000e+00, -3.05175781e-05]),
  'sampling_rate': 16000},
 'transcriptions': 'अब अपने keyboard पर एक साथ ctrl + alt + t कीज़ दबाकर terminal खोलते हैं'}

In [20]:
def prepare_dataset(batch):
    audio = batch["audio_paths"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcriptions"]).input_ids
    return batch


In [21]:
test_waud = test_waud.map(prepare_dataset, cache_file_name="/scratch/work/palp3/MUCS/dataset/test_dataset_cache.arrow", num_proc=None) #accessible loc: /scratch/elec/puhe/p/palp3/MUCS/test_dataset_cache_v2.arrow

In [22]:
train_waud = train_waud.map(prepare_dataset, cache_file_name="/scratch/work/palp3/MUCS/dataset/train_dataset_cache.arrow", num_proc=None) #accessible loc: /scratch/elec/puhe/p/palp3/MUCS/train_dataset_cache_v2.arrow

In [23]:
test_waud

Dataset({
    features: ['audio_id', 'audio_paths', 'transcriptions', 'input_values', 'labels'],
    num_rows: 3136
})

In [22]:
test_waud["transcriptions"][1698]

'हम पहले ही देख चुके हैं कि dot odt document को dot pdf file में कैसे बदलें'

In [24]:
test_waud["audio_paths"][1698]

{'path': '/scratch/work/palp3/MUCS/dataset/test_segmented/118638_nJfFvNUB8tQkaGgp_0024.wav',
 'array': array([2.44140625e-04, 2.13623047e-04, 2.13623047e-04, ...,
        4.96673584e-01, 5.58654785e-01, 6.13220215e-01]),
 'sampling_rate': 16000}

In [10]:
test_waud["audio_paths"][0]["array"]

array([ 0.00000000e+00, -3.05175781e-05,  0.00000000e+00, ...,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [15]:
test_waud["transcriptions"]#["array"]

['लिबर ऑफिस impress में एक प्रस्तुति document बनाना और बुनियादी formatting के इस spoken tutorial में आपका स्वागत है',
 'इस tutorial में हम impress window के भागों के बारे में सीखेंगे और कैसे स्लाइड इन्सर्ट करें और कॉपी करें फॉन्ट तथा फॉन्ट को फॉर्मेट करना सीखेंगे',
 'यहाँ हम अपने ऑपरेटिंग सिस्टम के रूप में gnu/linux और लिबरऑफिस वर्जन 334 का उपयोग कर रहे हैं',
 'चलिए अपनी प्रस्तुति प्रेजैटेशन sample impress open करते हैं जिसे पिछले tutorial में बनाया था',
 'चलिए देखते हैं कि screen पर क्या क्या है',
 'मध्य में हम खाली जगह देखते है जोकि workspace है जहाँ हम काम करेंगे',
 'जैसे कि आप देख सकते हैं workspace में 5 tabs हैं जिन्हें view buttons कहते हैं',
 'फिलहाल normal टैब चुनित हैं',
 'यह मुख्य view है अलगअलग slides बनाने के लिए',
 'outline view प्रत्येक slide के outline format में विषय शीर्षक bulleted और क्रमांकित सूचियाँ बताता है',
 'notes view प्रत्येक slide में notes जोड़ने की सुविधा देता है जोकि प्रस्तुतिकरण के वक्त नज़र नहीं the',
 'handout view slides को हैन्डाउट के रूप में print क

In [18]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(test_waud["transcriptions"]))
print(rand_int)
print(test_waud["transcriptions"][rand_int])
print(test_waud["audio_id"][rand_int])
print(test_waud["audio_paths"][rand_int])

ipd.Audio(data=np.asarray(test_waud["audio_paths"][rand_int]["array"]), autoplay=True, rate=16000)

1178
untitled documents के साथ अन्य window पर जाने के लिए sample c से वापस जायें
508088_RRrEfkw4Rh5TdJkS_0076.wav
{'path': '/scratch/work/palp3/MUCS/dataset/test_segmented/508088_RRrEfkw4Rh5TdJkS_0076.wav', 'array': array([ 2.46887207e-02, -1.04064941e-02, -2.71911621e-02, ...,
        0.00000000e+00, -3.05175781e-05,  0.00000000e+00]), 'sampling_rate': 16000}
